In [51]:
%%writefile app.py
import streamlit as st
from pymongo import MongoClient
import pandas as pd

def mongo_to_df(mongo_list):
    """Convert MongoDB documents to pandas DataFrame with string _id."""
    df = pd.DataFrame(mongo_list)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df

@st.cache_data
def get_data():
    """Fetch students, courses, and enrollments from MongoDB."""
    client = MongoClient("mongodb://localhost:27017/")
    db = client["schools_db"] 

    students = list(db.students.find())
    courses = list(db.courses.find())
    enrollments = list(db.enrollments.find())

    return students, courses, enrollments

st.set_page_config(page_title="School Dashboard", layout="wide")
st.title(" Streamlit + MongoDB Dashboard")
st.write("**Assignment Q1 - Data Connection & Display**")

view_mode = st.sidebar.radio(
    "Select View Mode:",
    ("Tabular View", "Raw JSON View")
)

students, courses, enrollments = get_data()

tab1, tab2, tab3 = st.tabs(["Students", "Courses", "Enrollments"])

with tab1:
    st.subheader("Students Data")
    df_students = mongo_to_df(students)
    if view_mode == "Tabular View":
        st.dataframe(df_students)
    else:
        st.json(students[:5])  

with tab2:
    st.subheader("Courses Data")
    df_courses = mongo_to_df(courses)
    if view_mode == "Tabular View":
        st.table(df_courses)
    else:
        st.json(courses[:5])

with tab3:
    st.subheader("Enrollments Data")
    df_enrollments = mongo_to_df(enrollments)
    if view_mode == "Tabular View":
        st.dataframe(df_enrollments)
    else:
        st.json(enrollments[:5])


Overwriting app.py


In [52]:
%%writefile app.py
import streamlit as st
from pymongo import MongoClient
import pandas as pd

def mongo_to_df(mongo_list):
    df = pd.DataFrame(mongo_list)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df

@st.cache_data
def get_data():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["schools_db"]
    students = list(db.students.find())
    courses = list(db.courses.find())
    enrollments = list(db.enrollments.find())
    return students, courses, enrollments

st.set_page_config(page_title="School Dashboard", layout="wide")
st.title(" Streamlit + MongoDB Dashboard")
st.write("**Assignment Q1 & Q2 - Data Display & Interactive Filtering**")

view_mode = st.sidebar.radio("Select View Mode:", ("Tabular View", "Raw JSON View"))

students, courses, enrollments = get_data()
df_students = mongo_to_df(students)
df_courses = mongo_to_df(courses)
df_enrollments = mongo_to_df(enrollments)

tab1, tab2, tab3 = st.tabs(["Students", "Courses", "Enrollments"])

with tab1:
    st.subheader("Students Data")
    if view_mode == "Tabular View":
        st.dataframe(df_students)
    else:
        st.json(students[:5])

with tab2:
    st.subheader("Courses Data")
    if view_mode == "Tabular View":
        st.table(df_courses)
    else:
        st.json(courses[:5])

with tab3:
    st.subheader("Enrollments Data")
    if view_mode == "Tabular View":
        st.dataframe(df_enrollments)
    else:
        st.json(enrollments[:5])

st.sidebar.subheader(" Filter Students & Enrollments")

course_col = "course_name" if "course_name" in df_courses.columns else df_courses.columns[0]
instructor_col = "instructor" if "instructor" in df_courses.columns else df_courses.columns[1]

course_options = df_courses[course_col].fillna("Unknown").unique().tolist()
instructor_options = df_courses[instructor_col].fillna("Unknown").unique().tolist()
grade_options = df_enrollments["grade"].fillna("N/A").unique().tolist() if "grade" in df_enrollments.columns else ["N/A"]

ages = df_students["age"].fillna(0).astype(int) if "age" in df_students.columns else pd.Series([0])
min_age, max_age = ages.min(), ages.max()

selected_courses = st.sidebar.multiselect("Select Course(s):", course_options, default=course_options)
selected_grades = st.sidebar.multiselect("Select Grade(s):", grade_options, default=grade_options)
age_range = st.sidebar.slider("Select Age Range:", int(min_age), int(max_age), value=(int(min_age), int(max_age)))
selected_instructors = st.sidebar.multiselect("Select Instructor(s):", instructor_options, default=instructor_options)

df_merged = df_enrollments.merge(df_students, on="student_id", how="left") \
                          .merge(df_courses, on="course_id", how="left")

filtered = df_merged[
    (df_merged[course_col].isin(selected_courses)) &
    (df_merged["grade"].isin(selected_grades)) &
    (df_merged["age"].between(age_range[0], age_range[1])) &
    (df_merged[instructor_col].isin(selected_instructors))
]

st.subheader("Filtered Enrollments")
if filtered.empty:
    st.warning("No data found!")
else:
    columns_to_show = ["student_id", "name", "age", "grade", "course_id", course_col, instructor_col]
    columns_to_show = [c for c in columns_to_show if c in filtered.columns]
    st.dataframe(filtered[columns_to_show])

# Show applied filter
query_dict = {
    "courses": selected_courses,
    "grades": selected_grades,
    "age_range": age_range,
    "instructors": selected_instructors
}

with st.expander("Applied Filter"):
    st.code(query_dict)


Overwriting app.py


In [53]:
%%writefile app.py
import streamlit as st
from pymongo import MongoClient
import pandas as pd

def mongo_to_df(mongo_list):
    df = pd.DataFrame(mongo_list)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df

client = MongoClient("mongodb://localhost:27017/")
db = client["schools_db"]

@st.cache_data
def get_data():
    students = list(db.students.find())
    courses = list(db.courses.find())
    enrollments = list(db.enrollments.find())
    return students, courses, enrollments

st.set_page_config(page_title="School Dashboard", layout="wide")
st.title("Streamlit + MongoDB Dashboard")
st.write("**Assignment Q1-Q3: Data, Filtering, KPIs & Editing**")

students, courses, enrollments = get_data()
df_students = mongo_to_df(students)
df_courses = mongo_to_df(courses)
df_enrollments = mongo_to_df(enrollments)

view_mode = st.sidebar.radio("Select View Mode:", ("Tabular View", "Raw JSON View"))
tab1, tab2, tab3 = st.tabs(["Students", "Courses", "Enrollments"])

with tab1:
    st.subheader("Students Data")
    if view_mode == "Tabular View":
        st.dataframe(df_students)
    else:
        st.json(students[:5])

with tab2:
    st.subheader("Courses Data")
    if view_mode == "Tabular View":
        st.table(df_courses)
    else:
        st.json(courses[:5])

with tab3:
    st.subheader("Enrollments Data")
    if view_mode == "Tabular View":
        st.dataframe(df_enrollments)
    else:
        st.json(enrollments[:5])

st.sidebar.subheader(" Filter Students & Enrollments")

course_col = "course_name"
instructor_col = "instructor"

course_options = df_courses[course_col].fillna("Unknown").unique().tolist() if course_col in df_courses.columns else ["Unknown"]
instructor_options = df_courses[instructor_col].fillna("Unknown").unique().tolist() if instructor_col in df_courses.columns else ["Unknown"]
grade_options = df_enrollments["grade"].fillna("N/A").unique().tolist() if "grade" in df_enrollments.columns else ["N/A"]

ages = df_students["age"].fillna(0).astype(int) if "age" in df_students.columns else pd.Series([0])
min_age, max_age = ages.min(), ages.max()

selected_courses = st.sidebar.multiselect("Select Course(s):", course_options, default=course_options)
selected_grades = st.sidebar.multiselect("Select Grade(s):", grade_options, default=grade_options)
age_range = st.sidebar.slider("Select Age Range:", int(min_age), int(max_age), value=(int(min_age), int(max_age)))
selected_instructors = st.sidebar.multiselect("Select Instructor(s):", instructor_options, default=instructor_options)

df_merged = df_enrollments.merge(df_students, on="student_id", how="left") \
                          .merge(df_courses, on="course_id", how="left")

filtered = df_merged[
    (df_merged[course_col].isin(selected_courses)) &
    (df_merged["grade"].isin(selected_grades)) &
    (df_merged["age"].between(age_range[0], age_range[1])) &
    (df_merged[instructor_col].isin(selected_instructors))
]

st.subheader("Filtered Enrollments")
if filtered.empty:
    st.warning("No data found!")
else:
    columns_to_show = ["student_id", "name", "age", "grade", "course_id", course_col, instructor_col]
    columns_to_show = [c for c in columns_to_show if c in filtered.columns]
    st.dataframe(filtered[columns_to_show])

with st.expander("Applied Filter"):
    st.code({
        "courses": selected_courses,
        "grades": selected_grades,
        "age_range": age_range,
        "instructors": selected_instructors
    })

st.subheader(" Key Performance Indicators (KPIs)")
col1, col2, col3, col4 = st.columns(4)

total_students = filtered["student_id"].nunique() if not filtered.empty else 0
total_courses = filtered["course_id"].nunique() if not filtered.empty else 0
avg_age = round(filtered["age"].mean(), 1) if not filtered.empty else 0
enroll_count = len(filtered) if not filtered.empty else 0

col1.metric("Total Students", total_students)
col2.metric("Total Courses", total_courses)
col3.metric("Avg Age", avg_age)
col4.metric("Enrollments Count", enroll_count)

# Top 5 popular courses
if not filtered.empty:
    top_courses = filtered.groupby(course_col).size().sort_values(ascending=False).head(5)
    with st.expander("More Insights: Top 5 Courses"):
        st.bar_chart(top_courses)

st.subheader(" Edit Student Info (Grade)")
if not filtered.empty:
    editable_cols = ["student_id", "name", "grade"]
    editable_df = filtered[editable_cols].drop_duplicates(subset=["student_id"])
    edited_df = st.data_editor(editable_df, num_rows="dynamic")

    if st.button("Update Grades in MongoDB"):
        for _, row in edited_df.iterrows():
            db.students.update_one({"student_id": row["student_id"]}, {"$set": {"grade": row["grade"]}})
        st.success("Grades updated successfully!")


Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
from pymongo import MongoClient

def mongo_to_df(mongo_list):
    df = pd.DataFrame(mongo_list)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df

@st.cache_data
def get_data():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["schools_db"]
    students = list(db.students.find())
    courses = list(db.courses.find())
    enrollments = list(db.enrollments.find())
    return students, courses, enrollments

st.set_page_config(page_title="Q4 - Advanced Visualizations", layout="wide")
st.title(" Question 4 - Advanced Visualizations")

students, courses, enrollments = get_data()
df_students = mongo_to_df(students)
df_courses = mongo_to_df(courses)
df_enrollments = mongo_to_df(enrollments)

if "age" not in df_students.columns:
    df_students["age"] = 0
else:
    df_students["age"] = df_students["age"].fillna(0).astype(int)

if "grade" not in df_enrollments.columns:
    df_enrollments["grade"] = "N/A"
else:
    df_enrollments["grade"] = df_enrollments["grade"].fillna("N/A")

if "course_name" not in df_courses.columns:
    df_courses["course_name"] = df_courses.get("title", "Unknown").fillna("Unknown")
else:
    df_courses["course_name"] = df_courses["course_name"].fillna("Unknown")

if "enrollment_date" not in df_enrollments.columns:
    df_enrollments["enrollment_date"] = pd.NaT
else:
    df_enrollments["enrollment_date"] = pd.to_datetime(df_enrollments["enrollment_date"], errors="coerce")

df_merged = df_enrollments.merge(df_students, on="student_id", how="left") \
                          .merge(df_courses, left_on="course_id", right_on="course_id", how="left")

if "grade" not in df_merged.columns:
    possible_cols = [c for c in df_merged.columns if "grade" in c.lower()]
    if possible_cols:
        df_merged["grade"] = df_merged[possible_cols[0]].fillna("N/A")
    else:
        df_merged["grade"] = "N/A"

tab1, tab2, tab3 = st.tabs(["Enrollment Trends", "Grade Distribution", "Course Popularity"])

with tab1:
    st.subheader("Enrollment Trends Over Time")
    if "enrollment_date" in df_merged.columns and not df_merged["enrollment_date"].isnull().all():
        trends = df_merged.groupby(df_merged['enrollment_date'].dt.to_period("M")).size().reset_index(name='count')
        trends['enrollment_date'] = trends['enrollment_date'].dt.to_timestamp()
        fig = px.line(trends, x='enrollment_date', y='count', title="Monthly Enrollment Trends")
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("No 'enrollment_date' data available for trend analysis.")

with tab2:
    st.subheader("Number of Students per Grade")
    grade_counts = df_merged.groupby("grade")["student_id"].nunique().reset_index(name="student_count")
    fig = px.bar(grade_counts, x='grade', y='student_count', color='student_count', title="Students per Grade")
    st.plotly_chart(fig, use_container_width=True)

with tab3:
    st.subheader("Students by Course")
    course_counts = df_merged.groupby("course_name")["student_id"].nunique().reset_index(name="student_count")
    fig = px.pie(course_counts, names='course_name', values='student_count', hole=0.4,
                 title="Course Popularity (Donut Chart)")
    st.plotly_chart(fig, use_container_width=True)


Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
from io import BytesIO

def mongo_to_df(mongo_list):
    df = pd.DataFrame(mongo_list)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df
st.set_page_config(page_title="School Dashboard", layout="wide")
st.title("School Dashboard")

with st.sidebar.form("login_form"):
    username = st.text_input("Enter your username", "root")
    submitted = st.form_submit_button("Login")
if submitted:
    st.success(f"Hi, {username}! Welcome to the School Dashboard.")

with st.spinner("Loading data from MongoDB..."):
    client = MongoClient("mongodb://localhost:27017/")
    db = client["schools_db"]
    students = list(db.students.find())
    courses = list(db.courses.find())
    enrollments = list(db.enrollments.find())

df_students = mongo_to_df(students)
df_courses = mongo_to_df(courses)
df_enrollments = mongo_to_df(enrollments)

if "age" not in df_students.columns:
    df_students["age"] = 0
if "course_name" not in df_courses.columns:
    df_courses["course_name"] = "Unknown"
if "instructor" not in df_courses.columns:
    df_courses["instructor"] = "Unknown"

df_merged = df_enrollments.merge(df_students, on="student_id", how="left") \
                          .merge(df_courses, on="course_id", how="left")

grade_cols = [c for c in df_merged.columns if "grade" in c.lower()]
if grade_cols:
    grade_col = grade_cols[0]
    df_merged[grade_col] = df_merged[grade_col].fillna("N/A")
else:
    grade_col = "grade"
    df_merged[grade_col] = "N/A"

st.sidebar.subheader(" Filter Students & Enrollments")

course_col = "course_name"
instructor_col = "instructor"

course_options = df_merged[course_col].dropna().unique().tolist()
instructor_options = df_merged[instructor_col].dropna().unique().tolist()
grade_options = df_merged[grade_col].dropna().unique().tolist()

min_age = int(df_merged["age"].min()) if "age" in df_merged.columns else 0
max_age = int(df_merged["age"].max()) if "age" in df_merged.columns else 100

selected_courses = st.sidebar.multiselect("Select Course(s):", course_options, default=course_options)
selected_grades = st.sidebar.multiselect("Select Grade(s):", grade_options, default=grade_options)
age_range = st.sidebar.slider("Select Age Range:", min_age, max_age, value=(min_age, max_age))
selected_instructors = st.sidebar.multiselect("Select Instructor(s):", instructor_options, default=instructor_options)

filtered = df_merged[
    (df_merged[course_col].isin(selected_courses)) &
    (df_merged[grade_col].isin(selected_grades)) &
    (df_merged["age"].between(age_range[0], age_range[1])) &
    (df_merged[instructor_col].isin(selected_instructors))
]

st.subheader("Filtered Enrollments")
if filtered.empty:
    st.warning("No data found!")
else:
    st.dataframe(filtered)

with st.expander("Applied Filter"):
    st.code({
        "courses": selected_courses,
        "grades": selected_grades,
        "age_range": age_range,
        "instructors": selected_instructors
    })

st.subheader(" Key Metrics")
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Students", filtered["student_id"].nunique())
col2.metric("Total Courses", filtered[course_col].nunique())
col3.metric("Average Age", round(filtered["age"].mean(), 1) if not filtered.empty else 0)
col4.metric("Enrollments Count", len(filtered))

# More insights
with st.expander("More insights: Top 5 popular courses"):
    top_courses = filtered.groupby(course_col)["student_id"].nunique().sort_values(ascending=False).head(5)
    st.table(top_courses.reset_index().rename(columns={"student_id": "Number of Students"}))

st.subheader("Edit Student Info")
if not filtered.empty:
    edited = st.data_editor(filtered[["student_id", "name", "age", grade_col]], num_rows="dynamic")
    if st.button("Update Changes"):
        for _, row in edited.iterrows():
            db.students.update_one({"student_id": row["student_id"]}, {"$set": {"age": row["age"], "grade": row.get(grade_col, "")}})
        st.success("Student records updated successfully!")

st.subheader(" Advanced Visualizations")
tab1, tab2, tab3 = st.tabs(["Enrollment Trends", "Grade Distribution", "Course Popularity"])

with tab1:
    st.markdown("**Enrollment Trends Over Time**")
    if "enrollment_date" in filtered.columns:
        filtered['enrollment_date'] = pd.to_datetime(filtered['enrollment_date'])
        trends = filtered.groupby(filtered['enrollment_date'].dt.to_period("M")).size().reset_index(name='count')
        trends['enrollment_date'] = trends['enrollment_date'].dt.to_timestamp()
        fig = px.line(trends, x='enrollment_date', y='count', title="Monthly Enrollment Trends")
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("No 'enrollment_date' column found.")

with tab2:
    st.markdown("**Number of Students per Grade**")
    grade_counts = filtered.groupby(grade_col)["student_id"].nunique().reset_index(name="student_count")
    fig = px.bar(grade_counts, x=grade_col, y="student_count", color="student_count", title="Students per Grade")
    st.plotly_chart(fig, use_container_width=True)

with tab3:
    st.markdown("**Students by Course**")
    course_counts = filtered.groupby(course_col)["student_id"].nunique().reset_index(name="student_count")
    fig = px.pie(course_counts, names=course_col, values="student_count", hole=0.4, title="Course Popularity")
    st.plotly_chart(fig, use_container_width=True)

if not filtered.empty:
    csv_bytes = filtered.to_csv(index=False).encode()
    st.download_button(
        label=" Download Filtered Students as CSV",
        data=csv_bytes,
        file_name="filtered_students.csv",
        mime="text/csv"
    )


Overwriting app.py
